## Introduction
Functions are the backbone of Python programming. While you've learned the basics (parameters, return values, *args, **kwargs), there's much more to explore! Today, we'll dive deep into advanced concepts that professional Python developers use daily.

### Learning Objectives:
- Understand variable scope and namespaces
- Master default and keyword arguments
- Explore *args and **kwargs in depth
- Learn about decorators (function wrappers)
- Understand higher-order functions
- Apply functional programming concepts
- Real-world function optimization

## 1. Understanding Scope and Namespaces
Scope determines where a variable can be accessed and modified

In [ ]:
# Global Scope - visible everywhere in the module
global_var = "I am global"

def outer_function():
    # Local Scope - only visible inside outer_function
    outer_var = "I am in outer"
    
    def inner_function():
        # Nested Scope - can access outer's variables
        inner_var = "I am in inner"
        print(f"Inner: {inner_var}")
        print(f"Outer (from inner): {outer_var}")
        print(f"Global (from inner): {global_var}")
    
    inner_function()
    # print(inner_var)  # Would cause NameError - inner_var not accessible here

outer_function()
# print(outer_var)  # Would cause NameError - outer_var not accessible here
print(f"Global: {global_var}")


In [ ]:
# LEGB Rule: Local, Enclosing, Global, Built-in
x = "Global"

def outer():
    x = "Enclosing"  # Enclosing scope
    
    def inner():
        x = "Local"   # Local scope
        print(f"Local: {x}")
    
    inner()
    print(f"Outer function sees: {x}")

outer()

# Name shadowing - local variable hides global
global_value = 100

def example():
    global_value = 50  # This creates a local variable, not modifying global
    print(f"Inside function: {global_value}")

example()
print(f"After function: {global_value}")  # Still 100


### Using global and nonlocal Keywords


In [ ]:
# global keyword - modify global variable inside function
counter = 0

def increment_global():
    global counter  # Declare that we want to modify the global counter
    counter += 1
    print(f"Counter: {counter}")

increment_global()  # Counter: 1
increment_global()  # Counter: 2
print(f"Global counter: {counter}")  # 2

print("\n" + "="*40 + "\n")

# nonlocal keyword - modify enclosing function's variable
def make_multiplier(factor):
    multiplier = factor
    
    def multiply(x):
        nonlocal multiplier  # Modify the enclosing variable
        multiplier += 1  # This modifies the outer multiplier
        return x * multiplier
    
    return multiply

double = make_multiplier(2)
print(f"First call: {double(5)}")   # 5 * 3 = 15
print(f"Second call: {double(5)}")  # 5 * 4 = 20

# Without nonlocal - would create local variable
def without_nonlocal():
    x = 10
    
    def inner():
        x = 20  # Creates new local x, doesn't modify outer
        return x
    
    print(f"Inner returns: {inner()}")
    print(f"Outer x is still: {x}")

without_nonlocal()


## 2. Default and Keyword Arguments Mastery


In [ ]:
# Mutable Default Arguments - Common Pitfall!
def append_to_list(item, target_list=[]):  # ❌ DON'T DO THIS!
    target_list.append(item)
    return target_list

print("Problem with mutable defaults:")
print(append_to_list(1))      # [1]
print(append_to_list(2))      # [1, 2] - Same list!
print(append_to_list(3))      # [1, 2, 3] - Still the same!

print("\n" + "="*40 + "\n")

# Correct approach - use None as default
def append_to_list_correct(item, target_list=None):  # ✅ CORRECT
    if target_list is None:
        target_list = []
    target_list.append(item)
    return target_list

print("Correct approach:")
print(append_to_list_correct(1))  # [1]
print(append_to_list_correct(2))  # [2] - New list
print(append_to_list_correct(3))  # [3] - New list

print("\n" + "="*40 + "\n")

# Keyword-only arguments (after *)
def create_user(name, age, *, email, role="user"):
    """
    name, age are positional
    email is keyword-only (required)
    role is keyword-only (with default)
    """
    return {
        "name": name,
        "age": age,
        "email": email,
        "role": role
    }

user1 = create_user("Alice", 25, email="alice@example.com")
print(f"User 1: {user1}")

user2 = create_user("Bob", 30, email="bob@example.com", role="admin")
print(f"User 2: {user2}")

# This would cause an error:
# create_user("Charlie", 28, "charlie@example.com")  # TypeError - email must be keyword


## 3. Deep Dive: *args and **kwargs


In [ ]:
# *args - Variable number of positional arguments (as tuple)
def sum_all(*args):
    print(f"Args type: {type(args)}")
    print(f"Args content: {args}")
    return sum(args)

print("Using *args:")
print(sum_all(1, 2, 3))           # 6
print(sum_all(1, 2, 3, 4, 5))     # 15

print("\n" + "="*40 + "\n")

# **kwargs - Variable number of keyword arguments (as dictionary)
def print_config(**kwargs):
    print(f"Config type: {type(kwargs)}")
    for key, value in kwargs.items():
        print(f"  {key}: {value}")

print("Using **kwargs:")
print_config(host="localhost", port=8000, debug=True, db="sqlite")

print("\n" + "="*40 + "\n")

# Combining everything: positional, *args, **kwargs
def flexible_function(first, second, *args, **kwargs):
    print(f"First: {first}")
    print(f"Second: {second}")
    print(f"Extra positional: {args}")
    print(f"Extra keyword: {kwargs}")

flexible_function(1, 2, 3, 4, 5, name="Alice", age=25)

print("\n" + "="*40 + "\n")

# Unpacking with * and **
numbers = [1, 2, 3, 4, 5]
print(f"Sum using unpacking: {sum_all(*numbers)}")  # Same as sum_all(1, 2, 3, 4, 5)

config = {"host": "localhost", "port": 8000, "debug": True}
print("\nConfig using unpacking:")
print_config(**config)  # Same as print_config(host="localhost", port=8000, debug=True)


## 4. Function Decorators - The Power of Higher-Order Functions
Decorators allow you to wrap a function and modify its behavior without changing the original function


In [ ]:
# Step 1: Understanding Closures (functions returning functions)
def make_greeter(greeting):
    """Outer function - creates a closure"""
    def greet(name):
        """Inner function - closes over 'greeting'"""
        return f"{greeting}, {name}!"
    return greet

# Create different greeters
say_hello = make_greeter("Hello")
say_goodbye = make_greeter("Goodbye")

print(say_hello("Alice"))      # Hello, Alice!
print(say_goodbye("Bob"))      # Goodbye, Bob!

print("\n" + "="*40 + "\n")

# Step 2: Simple Decorator
def my_decorator(func):
    """A simple decorator function"""
    def wrapper():
        print("Something before the function")
        func()
        print("Something after the function")
    return wrapper

@my_decorator
def say_hello_world():
    print("Hello, World!")

say_hello_world()

print("\n" + "="*40 + "\n")

# Step 3: Decorator with Arguments
def my_decorator_with_args(func):
    """Decorator that preserves function arguments"""
    def wrapper(*args, **kwargs):
        print(f"Calling {func.__name__} with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"Function returned: {result}")
        return result
    return wrapper

@my_decorator_with_args
def add(a, b):
    """Add two numbers"""
    return a + b

result = add(3, 5)

print("\n" + "="*40 + "\n")

# Step 4: Practical Decorator Example - Timing decorator
import time

def timer_decorator(func):
    """Measure execution time of a function"""
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{func.__name__} took {end - start:.4f} seconds")
        return result
    return wrapper

@timer_decorator
def slow_function():
    time.sleep(1)
    return "Done!"

result = slow_function()


## 5. Higher-Order Functions
Functions that take other functions as arguments or return functions


In [ ]:
# map() - Apply function to each element
numbers = [1, 2, 3, 4, 5]
squared = list(map(lambda x: x**2, numbers))
print(f"Squared numbers: {squared}")

# map with custom function
def double(x):
    return x * 2

doubled = list(map(double, numbers))
print(f"Doubled numbers: {doubled}")

print("\n" + "="*40 + "\n")

# filter() - Keep elements where function returns True
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
evens = list(filter(lambda x: x % 2 == 0, numbers))
print(f"Even numbers: {evens}")

greater_than_5 = list(filter(lambda x: x > 5, numbers))
print(f"Numbers > 5: {greater_than_5}")

print("\n" + "="*40 + "\n")

# reduce() - Apply function cumulatively
from functools import reduce

numbers = [1, 2, 3, 4, 5]
product = reduce(lambda x, y: x * y, numbers)
print(f"Product of {numbers} = {product}")

# Equivalent: 1 * 2 * 3 * 4 * 5 = 120

concatenated = reduce(lambda x, y: x + "-" + str(y), map(str, numbers))
print(f"Concatenated: {concatenated}")

print("\n" + "="*40 + "\n")

# Combining map, filter, and reduce
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Find sum of squares of even numbers
result = reduce(
    lambda x, y: x + y,
    map(lambda x: x**2, filter(lambda x: x % 2 == 0, data))
)
print(f"Sum of squares of even numbers: {result}")
# Explanation: [2,4,6,8,10] -> [4,16,36,64,100] -> 220


## 6. Functional Programming Concepts
lambda, comprehensions, and functional paradigms


In [ ]:
# Lambda - Anonymous functions
# lambda arguments: expression

square = lambda x: x ** 2
print(f"Square of 5: {square(5)}")

add = lambda x, y: x + y
print(f"Add 3 and 4: {add(3, 4)}")

# Lambda with multiple arguments and conditions
max_value = lambda x, y: x if x > y else y
print(f"Max of 10 and 20: {max_value(10, 20)}")

print("\n" + "="*40 + "\n")

# List comprehension vs functional approach
numbers = [1, 2, 3, 4, 5]

# Traditional loop
result1 = []
for x in numbers:
    if x % 2 == 0:
        result1.append(x ** 2)

# List comprehension (more Pythonic)
result2 = [x**2 for x in numbers if x % 2 == 0]

# Functional (map + filter)
result3 = list(map(lambda x: x**2, filter(lambda x: x % 2 == 0, numbers)))

print(f"All three approaches give: {result1}")

print("\n" + "="*40 + "\n")

# Nested comprehensions
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

# Flatten matrix
flattened = [num for row in matrix for num in row]
print(f"Flattened: {flattened}")

# Matrix transpose
transposed = [[row[i] for row in matrix] for i in range(3)]
print(f"Transposed: {transposed}")

# Dictionary comprehension
numbers = [1, 2, 3, 4, 5]
squares_dict = {x: x**2 for x in numbers}
print(f"Squares dict: {squares_dict}")

# Set comprehension
unique_chars = {char for char in "hello world" if char != ' '}
print(f"Unique characters: {unique_chars}")


## 7. Mini Project: Data Processing Pipeline
Create a real-world data processing application using advanced functions


In [ ]:
# Project: Student Grade Processing System
from functools import reduce

class StudentGradeProcessor:
    def __init__(self):
        self.students = [
            {"name": "Alice", "grades": [85, 90, 88, 92]},
            {"name": "Bob", "grades": [78, 82, 80, 85]},
            {"name": "Charlie", "grades": [95, 93, 97, 94]},
            {"name": "Diana", "grades": [88, 91, 87, 90]},
        ]
    
    def calculate_average(self, grades):
        """Calculate average using reduce"""
        return reduce(lambda a, b: a + b, grades) / len(grades)
    
    def add_average_to_student(self, student):
        """Add average grade to student dict"""
        student["average"] = self.calculate_average(student["grades"])
        return student
    
    def grade_to_letter(self, average):
        """Convert average to letter grade"""
        if average >= 90:
            return "A"
        elif average >= 80:
            return "B"
        elif average >= 70:
            return "C"
        elif average >= 60:
            return "D"
        else:
            return "F"
    
    def process_grades(self):
        """Process all grades using functional approach"""
        # Step 1: Add average to each student
        students_with_avg = list(map(self.add_average_to_student, self.students))
        
        # Step 2: Add letter grade to each student
        for student in students_with_avg:
            student["letter_grade"] = self.grade_to_letter(student["average"])
        
        return students_with_avg
    
    def get_top_performers(self, students, threshold=90):
        """Filter students above threshold"""
        return list(filter(lambda s: s["average"] >= threshold, students))
    
    def generate_report(self):
        """Generate full report"""
        students_with_avg = self.process_grades()
        
        print("="*60)
        print("STUDENT GRADE REPORT")
        print("="*60)
        
        for student in students_with_avg:
            avg = student["average"]
            print(f"{student['name']:12} | Avg: {avg:5.2f} | Grade: {student['letter_grade']}")
        
        print("\n" + "-"*60)
        print("TOP PERFORMERS (90+)")
        print("-"*60)
        
        top_performers = self.get_top_performers(students_with_avg)
        for student in top_performers:
            print(f"  {student['name']}: {student['average']:.2f}")
        
        # Class average
        class_avg = reduce(
            lambda total, s: total + s["average"],
            students_with_avg,
            0
        ) / len(students_with_avg)
        
        print(f"\nClass Average: {class_avg:.2f}")

# Run the processor
processor = StudentGradeProcessor()
processor.generate_report()


## 8. Summary: Day 14 - Functions Advanced Level

### Key Concepts Covered:
1. **Scope & Namespaces** - Local, Enclosing, Global, Built-in (LEGB)
2. **global and nonlocal Keywords** - Modifying outer scope variables
3. **Default Arguments** - Mutable vs immutable defaults
4. **Keyword-Only Arguments** - Forcing keyword usage with *
5. ***args** - Variable positional arguments
6. ****kwargs** - Variable keyword arguments
7. **Closures** - Functions that capture outer scope
8. **Decorators** - Wrapping functions to modify behavior
9. **Higher-Order Functions** - map(), filter(), reduce()
10. **Lambda Functions** - Anonymous single-expression functions
11. **Comprehensions** - List, dict, set comprehensions
12. **Functional Programming** - Composing functions for data processing

### Common Mistakes to Avoid:
```python
# ❌ Wrong: Mutable default argument
def add_item(item, lst=[]):
    lst.append(item)
    return lst

# ✅ Correct: Use None
def add_item(item, lst=None):
    if lst is None:
        lst = []
    lst.append(item)
    return lst

# ❌ Wrong: Forgetting *args is a tuple
def print_args(*args):
    print(args[0])  # Works, but
    for arg in args: print(arg)

# ✅ Correct: Use as tuple from the start
def print_args(*args):
    for arg in args:
        print(arg)

# ❌ Wrong: Not using @wraps in decorators
def my_decorator(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

# ✅ Correct: Preserve metadata
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper
```

### Best Practices:
1. Use keyword-only arguments for clarity in function signatures
2. Avoid mutable default arguments
3. Use list comprehensions instead of map/filter when possible (more Pythonic)
4. Write small, focused decorators
5. Use type hints for complex functions
6. Document your functions with docstrings
7. Keep functions pure (no side effects when possible)

### Real-World Applications:
- **Web Frameworks**: Decorators for routing (@app.route)
- **Authentication**: Decorators for access control
- **Data Processing**: map/filter/reduce for data transformation
- **Configuration**: Functional argument builders
- **API Wrappers**: Decorators for logging, retry logic, caching
- **Testing**: Decorators for test fixtures and mocking
- **Performance**: Timing and profiling decorators

### Performance Notes:
- Comprehensions are faster than map/filter for most cases
- Decorators add minimal overhead
- Closures maintain reference to outer scope (memory consideration)
- Lambda functions have slight overhead vs named functions

---

## Day 15 Preview: Object-Oriented Programming - Classes & Objects
Next session, we'll cover:
- Creating classes and objects
- Instance and class variables
- Methods and self
- Constructors (__init__)
- Inheritance and polymorphism
- Magic methods (__str__, __repr__, __add__, etc.)
- Object-oriented design principles
